**Notebook preparation and importing libraries**

In [1]:
import warnings
from hvac import Quantity
from hvac.fluids import HumidAir, Fluid, CP_HUMID_AIR, CoolPropWarning

Turn off the warnings that come from module `hvac.fluids.fluid`:

In [2]:
warnings.filterwarnings('ignore', category=CoolPropWarning)

In [3]:
Q_ = Quantity

# Preliminary Heat Exchanger Calculations

- An air stream must be cooled from 24 °C (RH 50%) to 13 °C. The mass flow rate of air is 1500 kg/hr.
- The refrigerant is R134a.
- The outdoor air temperature is 35 °C.

Define the type of refrigerant:

In [4]:
R134a = Fluid('R134a')

Define the air state at the entrance and exit of the evaporator:

In [5]:
evp_air_in = HumidAir(Tdb=Q_(24, 'degC'), RH=Q_(50, 'pct'))
evp_air_out = HumidAir(Tdb=Q_(13, 'degC'), RH=Q_(100, 'pct'))

It is assumed that air will be close to saturation at the exit of the evaporator.

Set the mass flow rate of air through the evaporator:

In [6]:
evp_m_dot_air = Q_(1500, 'kg/hr')

## Heat absorption rate

The heat rate to be extracted from the air stream in the evaporator can now be calculated:

In [7]:
Q_evp = evp_m_dot_air * (evp_air_in.h - evp_air_out.h)
print(f"{Q_evp.to('kW'):~P.3f}")

4.659 kW


## Evaporation temperature and pressure

Select the evaporation temperature:

In [8]:
T_evp = Q_(5, 'degC')

Find evaporation pressure. The state of saturated vapor in the evaporator:

In [9]:
evp_rfg_sat_vap = R134a(T=T_evp, x=Q_(1.0, 'frac'))
P_evp = evp_rfg_sat_vap.P
print(f"evaporation pressure = {P_evp.to('bar'):~P.3f}")

evaporation pressure = 3.497 bar


## Condensation temperature and pressure

Select the condensing temperature:

In [10]:
T_cnd = Q_(50, 'degC')

Find condensing pressure. The state of saturated liquid in the condenser:

In [11]:
cnd_rfg_sat_liq = R134a(T=T_cnd, x=Q_(0.0, 'frac'))
P_cnd = cnd_rfg_sat_liq.P
print(f"condensing pressure = {P_cnd.to('bar'):~P.3f}")

condensing pressure = 13.179 bar


## Mass flow rate of refrigerant

First, find the state of refrigerant leaving and entering the evaporator. Refrigerant leaving the evaporator is superheated to a certain degree, which is a setting of the expansion device. The state of refrigerant entering the evaporator will depend on the degree of subcooling of the refrigerant leaving the condenser.

Select the degree of superheating:

In [12]:
dT_sh = Q_(10, 'K')

Select the degree of subcooling:

In [13]:
dT_sc = Q_(5, 'K')

Find state of refrigerant leaving the evaporator:

In [14]:
evp_rfg_out = R134a(T=T_evp.to('K') + dT_sh, P=P_evp)

Find state of refrigerant entering the evaporator. The enthalpy of refrigerant entering the evaporator is equal to the enthalpy of refrigerant leaving the condenser, as the expansion process is considered to be an isenthalpic process. So first, the state of refrigerant leaving the condenser must be determined:

In [15]:
cnd_rfg_out = R134a(T=T_cnd.to('K') - dT_sc, x=Q_(0.0, 'frac'))

Now, the state of refrigerant entering the evaporator can also be determined:

In [16]:
evp_rfg_in = R134a(h=cnd_rfg_out.h, P=P_evp)
print(f"vapor quality of entering refrigerant to evaporator = {evp_rfg_in.x.to('frac'):~P.2f}")

vapor quality of entering refrigerant to evaporator = 0.29 frac


Now that the states of refrigerant on both sides of the evaporator are known, we can determine the mass flow rate of refrigerant needed to get at the required heat absorption rate in the evaporator:

In [17]:
m_dot_rfg = Q_evp / (evp_rfg_out.h - evp_rfg_in.h)
print(f"mass flow rate of refrigerant = {m_dot_rfg.to('kg/hr'):~P.3f}")

mass flow rate of refrigerant = 114.344 kg/h


## Heat rejection rate

The heat rate to be rejected in the condenser is the sum of the heat rate absorbed in the evaporator and the compressor power. To determine the compressor power, a compressor selection software program (e.g., Coolselector2 from Danfoss) can be used. Entering into this program the required heat absorption rate, the selected evaporation temperature, the selected degree of superheat, the selected condensing temperature, the selected degree of subcooling, and the refrigerant to be used, several compressor models will be presented to choose from. The program will also show the performance details of the currently selected compressor. For this example, a compressor DANFOSS VTZ038-G is selected. According to the selection program, the estimated discharge temperature is about 85.0 °C and the compressor power is about 1.855 kW. If we use the indicated refrigerant discharge temperature, we can determine the state of refrigerant entering the condenser. We already know the state of refrigerant leaving the condenser and the mass flow rate of refrigerant. With all this, the heat rejection rate can be calculated. Of course, we could also add the compressor power to the heat absorption rate to get at the heat rejection rate in the condenser.  

In [18]:
T_dis = Q_(85.0, 'degC')
cnd_rfg_in = R134a(T=T_dis, P=P_cnd)
Q_cnd = m_dot_rfg * (cnd_rfg_in.h - cnd_rfg_out.h)
print(f"heat rejection rate (based on discharge temperature) = {Q_cnd.to('kW'):~P.3f}")

heat rejection rate (based on discharge temperature) = 6.329 kW


If we add the indicated compressor power to the heat absorption rate, we get:

In [19]:
_Q_cnd = Q_evp + Q_(1.855, 'kW')
print(f"heat rejection rate (based on compressor power) = {_Q_cnd.to('kW'):~P.3f}")

heat rejection rate (based on compressor power) = 6.514 kW


## Mass flow rate of air through condenser

Refrigerant enters the condenser at a temperature around 85.0 °C, and it leaves the condenser at a temperature around 45 °C, since the condensing temperature is 50 °C and, we assumed 5 K of subcooling at the exit of the condenser. Air enters the condenser at 35 °C, where refrigerant leaves the condenser (counter-flow heat exchanger). If we allow a temperature rise of 10 K of the air stream through the condenser, we can determine the mass flow rate of air to accomplish this:

In [20]:
cnd_dT_air = Q_(10, 'K')
cnd_T_air_in = Q_(35, 'degC')
cnd_air_in = HumidAir(Tdb=cnd_T_air_in, RH=Q_(30, 'pct'))

cnd_T_air_out = cnd_T_air_in.to('K') + cnd_dT_air
cnd_air_out = HumidAir(Tdb=cnd_T_air_out, W=cnd_air_in.W)

cnd_m_dot_air = Q_cnd / (CP_HUMID_AIR * (cnd_T_air_out - cnd_T_air_in))
print(f"mass flow rate of air through condenser = {cnd_m_dot_air.to('kg / hr'):~P.3f}")

mass flow rate of air through condenser = 2233.861 kg/h


## Frontal area of evaporator

Select an air face velocity:

In [21]:
evp_v_fa = Q_(2, 'm/s')

The volume flow rate of air through the evaporator can be determined:

In [22]:
evp_V_dot_air = evp_m_dot_air / evp_air_in.rho
print(f"volume flow rate of air through evaporator = {evp_V_dot_air.to('m ** 3 / hr'):~P.3f}")

volume flow rate of air through evaporator = 1281.170 m³/h


The frontal area of the evaporator can now be calculated as:

In [23]:
evp_A_fr = evp_V_dot_air / evp_v_fa
print(f"frontal area of evaporator = {evp_A_fr.to('m ** 2'):~P.4f}")

frontal area of evaporator = 0.1779 m²


To determine the width and height of the frontal area, we could select a certain aspect ratio (ratio of height to width):

In [24]:
evp_AR = 1 / 3

The width of the frontal area:

In [25]:
evp_L1 = (evp_A_fr / evp_AR) ** 0.5
print(f"width of evaporator's frontal area = {evp_L1.to('mm'):~P.0f}")

width of evaporator's frontal area = 731 mm


The height of the frontal area:

In [26]:
evp_L3 = evp_A_fr / evp_L1
print(f"height of evaporator's frontal area = {evp_L3.to('mm'):~P.0f}")

height of evaporator's frontal area = 244 mm


## Frontal area of condenser

Select an air face velocity:

In [27]:
cnd_v_fa = Q_(2, 'm/s')

The volume flow rate of air through the condenser can be determined:

In [28]:
cnd_V_dot_air = cnd_m_dot_air / cnd_air_in.rho
print(f"volume flow rate of air through condenser = {cnd_V_dot_air.to('m ** 3 / hr'):~P.3f}")

volume flow rate of air through condenser = 1982.701 m³/h


The frontal area of the condenser can now be calculated as:

In [29]:
cnd_A_fr = cnd_V_dot_air / cnd_v_fa
print(f"frontal area of condenser = {cnd_A_fr.to('m ** 2'):~P.4f}")

frontal area of condenser = 0.2754 m²


To determine the width and height of the frontal area, we could select a certain aspect ratio (ratio of height to width):

In [30]:
cnd_AR = 1 / 3

The width of the frontal area:

In [31]:
cnd_L1 = (cnd_A_fr / cnd_AR) ** 0.5
print(f"width of condenser's frontal area = {cnd_L1.to('mm'):~P.0f}")

width of condenser's frontal area = 909 mm


The height of the frontal area:

In [32]:
cnd_L3 = cnd_A_fr / cnd_L1
print(f"height of condenser's frontal area = {cnd_L3.to('mm'):~P.0f}")

height of condenser's frontal area = 303 mm


## Summary

**Evaporator**

In [33]:
print(
    f"refrigerant in: {evp_rfg_in.T.to('degC'):~P.2f}, {evp_rfg_in.x.to('frac'):~P.2f}\n"
    f"air in: {evp_air_in.Tdb.to('degC'):~P.2f}, {evp_air_in.RH.to('pct'):~P.2f}\n"
    f"mass flow rate of air: {evp_m_dot_air.to('kg / hr'):~P.3f}\n"
    f"mass flow rate of refrigerant: {m_dot_rfg.to('kg / hr'):~P.3f}\n"
    f"frontal area: {evp_A_fr.to('m ** 2'):~P.4f}\n"
    f"frontal area width and height = {evp_L1.to('mm'):~P.0f}, {evp_L3.to('mm'):~P.0f}"
)

refrigerant in: 5.00 °C, 0.29 frac
air in: 24.00 °C, 50.00 %
mass flow rate of air: 1500.000 kg/h
mass flow rate of refrigerant: 114.344 kg/h
frontal area: 0.1779 m²
frontal area width and height = 731 mm, 244 mm


**Condenser**

In [34]:
print(
    f"refrigerant in: {cnd_rfg_in.T.to('degC'):~P.2f}, {cnd_rfg_in.P.to('bar'):~P.3f}\n"
    f"air in: {cnd_air_in.Tdb.to('degC'):~P.2f}, {cnd_air_in.RH.to('pct'):~P.2f}\n"
    f"mass flow rate of air: {cnd_m_dot_air.to('kg / hr'):~P.3f}\n"
    f"heat rejection rate: {Q_cnd.to('kW'):~P.3f}\n"
    f"frontal area: {cnd_A_fr.to('m ** 2'):~P.4f}\n"
    f"frontal area width and height = {cnd_L1.to('mm'):~P.0f}, {cnd_L3.to('mm'):~P.0f}"
)

refrigerant in: 85.00 °C, 13.179 bar
air in: 35.00 °C, 30.00 %
mass flow rate of air: 2233.861 kg/h
heat rejection rate: 6.329 kW
frontal area: 0.2754 m²
frontal area width and height = 909 mm, 303 mm
